### Get product info
- brand
- product name
- item number
- number of reviews
- number of loves
- price
- details/how to use/ingredients
- rating (number)
- category
- image

### Get review info
- reviewer name
- reviewer age
- reviewer is sephora employee
- reviewer other details
- review date
- review rating
- review title
- review text
- num not helpful/helpful

### Set up

In [136]:
from __future__ import print_function, division

# if needed: pip install requests or conda install requests
import requests

requests.__path__

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import chromedriver_binary

import pandas as pd
import datetime as dt
import random

import pickle
import json


In [137]:
# load reviews
def click_loop(counter=0):
    '''
    load reviews until:
    1. all reviews are loaded
    2. the button has been pressed 9 times (display 60 total)
    '''
    if counter==9:
        pass
    else:
        try:
            element = driver.find_element_by_class_name('css-1phfyoj')
            element.click()
            time.sleep(3)
            return click_loop(counter+1)
        except:
            pass

In [138]:
# main_url = 'https://www.sephora.com/product/alpha-beta-r-peel-extra-strength-daily-peel-mini-P425868?icid2=products%20grid:p425868:product'
# main_url = 'https://www.sephora.com/product/hydra-life-pores-away-pink-clay-mask-P417606' #30 reviews
# main_url = 'https://www.sephora.com/product/lotus-youth-preserve-dream-face-cream-P440312' #339 reviews
# main_url = 'https://www.sephora.com/product/protini-tm-polypeptide-cream-P427421' #3k reviews

In [139]:
# url_list = ['https://www.sephora.com/product/hydra-life-pores-away-pink-clay-mask-P417606',
#            'https://www.sephora.com/product/lotus-youth-preserve-dream-face-cream-P440312',
#            'https://www.sephora.com/product/protini-tm-polypeptide-cream-P427421']

In [140]:
# with open('pickles/link_list1.pickle','rb') as read_file:
#     url_list = pickle.load(read_file)

In [141]:
# driver.close()

In [142]:
# driver = webdriver.Chrome()

In [143]:
stored_errors = []

In [144]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")

driver = webdriver.Chrome(options=chrome_options)

In [145]:
for j,url in enumerate(url_list[11:]):
    try:
        start = time.time()
        driver.get(url)
        #give time for page to load
        time.sleep(10)
        #scroll to end
        # driver.find_element_by_tag_name('body').send_keys(Keys.END) 
        #click out of sign-up
        try:
            go_button = driver.find_element_by_xpath('//*[@id="modalDialog"]/div[2]/form/div[3]/div/div[1]/button')
            go_button.click()
        except:
            pass

        #go to reviews
#         driver.find_element_by_class_name('css-2rg6q7').click()
        driver.execute_script("document.getElementsByClassName('css-2rg6q7')[0].click()")
        time.sleep(5)
        #test rating pull - skip click_loop if it's missing
#         driver.find_element_by_class_name('css-2ljvdr').click()

        #loop through reviews
        click_loop()

        # collect data
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        # print(soup.prettify())

        # clean up item ID
        try:
            a = soup.find(class_='css-1qf1va5').text
            a1 = a[a.find('ITEM')+5:]
        except:
            a1 = 'Missing'

        # clean up details/how to use/ingredients
        try:
            details = soup.find_all(class_='css-pz80c5')[0].text
        except:
            details = 'N/A'
        try:
            how_to_use = soup.find_all(class_='css-pz80c5')[1].text
        except:
            how_to_use = 'N/A'
        try:
            ingredients = soup.find_all(class_='css-pz80c5')[2].text
        except:
            ingredients = 'N/A'

        # % recommend
        try:
            per_rec = soup.find(class_='css-1heqyf0').text
        except:
            per_rec = 'N/A'

        # initiate product dictionary
        prod_dict = {}
        prod_dict.update({'product_id':a1,
                          'url':url,
                          'brand':soup.find(class_='css-euydo4').text,
                          'name':soup.find(class_='css-1agrjfo').findNextSibling().text,
                          'review_count':soup.find_all(class_='css-2rg6q7')[0].text,
                          'love_count':soup.find_all(class_='css-2rg6q7')[1].text,
                          'price':soup.find(class_='css-14hdny6').text,
                          'details':details,
                          'how_to_use':how_to_use,
                          'ingredients':ingredients,
#                           'rating':soup.find(class_='css-1eqf5yr').text,
                          'rating':soup.find(class_='css-2ljvdr').text,
                          'percent_recommend':per_rec,
                          'categories':soup.find(class_='css-1doqpel').text,
                          'image_url':soup.find(class_='css-1ixbp0l').image['xlink:href']
                         })
        # prod_dict

        # initiate review dictionary
        review_dict = {}

        for i,_ in enumerate(soup.find_all(class_='css-1hm9c5d')[2:62]):
            # reviewer name
            try:
                name = _.find(class_ = 'css-y7wmlg').text
            except:
                name = 'No Name'  
            # clean up title - sometimes it is missing
            try:
                title = _.find(class_='css-ai9pjd').text
            except:
                title = 'No Title'
            # incentive
            try:
                incentive = _.find(class_='css-dl30gf').text
            except:
                incentive = 'N/A'
            # insider status
            try:
                insider = _.find(class_='css-n4wcj9')['alt']
            except:
                insider = 'None'
            # experience
            try:
                experience = _.find(class_='css-aemqch')['alt']
            except:
                experience = 'None'
            # reviewer details
            try:
                rev_details=[]
                for x in _.find_all(class_='css-15415he'):
                    rev_details.append(x.text)
            except:
                rev_details = []
            # recommends product
            try:
                rec = _.find(class_='css-ue839').text
            except:
                rec = 'N/A'


            # update dictionary
            review_dict.update({a1+"_"+str(i):{
                                'reviewer_name':name,
                                'incentive':incentive,
                                'insider':insider,
                                'experience':experience,
                                'reviewer_details':rev_details,
                                'review_date': _.find(class_ = 'css-1bk7s0g').text,
                                'review_rating': _.find(class_='css-5quttm')['aria-label'],
                                'review_title':title,
                                'review_text':_.find(class_='css-1p4f59m').text,
                                'review_not_helpful':_.find(class_ = 'css-39esqn').text[:15],
                                'review_helpful':_.find(class_ = 'css-39esqn').text[15:],
                                'recommends':rec
                                }})

        # review_dict

        #combine product and review dictionaries
        prod_dict['top_helpful_reviews'] = review_dict
        # prod_dict

        # save out dictionary
        with open('sephora_products/product'+str(a1)+'.json', 'w') as fp:
            json.dump(prod_dict, fp)
        
        # random lag 1
        time.sleep(.5+2*random.random())
        # random lag 2
        if (j+1 % 100 == 0):
            time.sleep(200+10*random.random())
        
        end = time.time()
        minutes = (end-start)/60
        minutes

        # product ID, number of scraped reviews, minutes it took, time finished
        meta = str(a1)+'|'+str(len(review_dict))+'|'+str(round(minutes,2))+'|'+str(dt.datetime.now())
        meta

        # save out tracker
        with open("sephora_products/tracker.txt", "a") as myfile:
            myfile.write(meta+'\n')
    except:
        stored_errors.append((j,url))
        print('missed one...'+url)
        raise
#         pass


missed one...https://www.sephora.com/product/t-l-c-framboos-tm-glycolic-night-serum-P392246


AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
driver.close()

In [ ]:
# print(soup.prettify())

In [ ]:
# url_list = []
# for i in stored_errors:
#     url_list.append(i[1])
# len(url_list)

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [147]:
print(soup.prettify())

<!DOCTYPE html>
<html class="css-1esi5of node-webkit landscape" data-comp="Index" lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head class="at-element-marker" data-comp="Head">
  <title>
   T.L.C. Framboos™ Glycolic Resurfacing Night Serum - Drunk Elephant | Sephora
  </title>
  <iframe src="javascript:void(0)" style="width: 0px; height: 0px; border: 0px; display: none;" title="">
  </iframe>
  <script src="//s.thebrighttag.com/tag?site=iEPEpth&amp;H=m9rtu4&amp;referrer=https%3A%2F%2Fwww.sephora.com%2Fproduct%2Ft-l-c-framboos-tm-glycolic-night-serum-P392246&amp;mode=v2&amp;cf=7407950" type="text/javascript">
  </script>
  <script src="//s.thebrighttag.com/tag?site=iEPEpth&amp;H=m9rtu4&amp;referrer=https%3A%2F%2Fwww.sephora.com%2Fproduct%2Ft-l-c-framboos-tm-glycolic-night-serum-P392246&amp;mode=v2&amp;cf=7407920" type="text/javascript">
  </script>
  <script src="//s.thebrighttag.com/tag?site=iEPEpth&amp;H=m9rtu4&amp;referrer=https%3A%2F%2Fwww.sephora.com%2Fproduct%2Ft-l-c-framboos-

In [146]:
url_list[11:]

['https://www.sephora.com/product/pure-one-step-camellia-cleansing-oil-P392235',
 'https://www.sephora.com/product/virgin-marula-tm-luxury-facial-oil-P392245',
 'https://www.sephora.com/product/t-l-c-framboos-tm-glycolic-night-serum-P392246',
 'https://www.sephora.com/product/hydra-therapy-skin-vitality-treatment-masks-P392627',
 'https://www.sephora.com/product/sunscreen-multi-protection-broad-spectrum-spf-50-P393065',
 'https://www.sephora.com/product/complexion-rescue-tinted-hydrating-gel-cream-P393356',
 'https://www.sephora.com/product/luna-sleeping-night-oil-P393718',
 'https://www.sephora.com/product/the-true-cream-aqua-bomb-P394639',
 'https://www.sephora.com/product/liplux-spf-30-P395625',
 'https://www.sephora.com/product/cc-creme-high-definition-radiance-P396095',
 'https://www.sephora.com/product/dermask-water-jet-vital-hydra-solution-P397623',
 'https://www.sephora.com/product/alpha-beta-ultra-gentle-daily-peel-sensitive-skin-P398761',
 'https://www.sephora.com/product/lum